<h1 style='font-size:40px'> Natural Language
Processing with RNNs and
Attention</h1>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O capítulo será composto por duas seções. Na primeira, iremos continuar desenvolvendo nossos estudos com RNN's, mas aplicadas no âmbito de NLP. Já na segunda, iremos dar enfoque aos mecanismos de atenção.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Generating Shakespearean Text Using a
Character RNN</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            Aqui, vamos montar uma rede capaz de gerar poemas com o estilo de escrita de Shakespeare. Ela será treinada para prever o próximo caractere a ser digitado.
        </li>
    </ul>
</div>

<p style='color:red'> Iniciei o capítulo; Creating the Training Dataset</p>

<p style='color:red'> Terminei de ler WaveNet. Começar Cap 16</p>